# Document Assistant - Interactive Demo

This notebook demonstrates the Document Assistant with LangGraph multi-agent architecture.

## Features Demonstrated:
- Intent Classification (qa, calculation, summarization)
- Multi-agent routing
- Tool usage (calculator, document_reader)
- Conversation memory
- Session persistence

## Setup

First, let's import the required modules and initialize the assistant.

In [1]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Import assistant modules
from src.assistant import DocumentAssistant
from src.config import get_default_llm

print("✅ Imports successful")
print(f"📁 Project root: {project_root}")

✅ Imports successful
📁 Project root: /Users/robertishimurasousa/Developer/GitHub/robertishimurasousa/doc_assistant_project


## Initialize LLM and Assistant

In [2]:
# Initialize LLM with Vocareum configuration
try:
    llm = get_default_llm()
    print("✅ LLM initialized successfully")
except ValueError as e:
    print(f"⚠️  Warning: {e}")
    print("Running without LLM - limited functionality")
    llm = None

# Initialize assistant with sample data
data_path = project_root / "data"
sessions_path = project_root / "sessions"  # Use absolute path to avoid creating notebooks/sessions

assistant = DocumentAssistant(
    document_path=str(data_path),
    session_dir=str(sessions_path),  # Explicitly set sessions directory
    llm=llm
)

# Load documents
count = assistant.load_documents()
print(f"\n📚 Loaded {count} documents from {data_path}")

# Display loaded documents
stats = assistant.get_stats()
print(f"\n📊 Statistics:")
print(f"  - Session ID: {stats['current_session_id']}")
print(f"  - Documents: {stats['num_documents']}")
print(f"  - Messages: {stats['current_session_messages']}")
print(f"  - Sessions directory: {sessions_path}")

✅ LLM initialized successfully
Loaded 6 document(s)

📚 Loaded 6 documents from /Users/robertishimurasousa/Developer/GitHub/robertishimurasousa/doc_assistant_project/data

📊 Statistics:
  - Session ID: None
  - Documents: 6
  - Messages: 0
  - Sessions directory: /Users/robertishimurasousa/Developer/GitHub/robertishimurasousa/doc_assistant_project/sessions


## Test 1: Q&A Intent (Information Retrieval)

The system should classify these queries as "qa" intent and route to the Q&A agent.

In [3]:
print("=" * 70)
print("TEST 1: Q&A Intent - Information Retrieval")
print("=" * 70)

queries = [
    "What was the total Q1 sales revenue?",
    "Who is the CTO of the company?",
    "What are the top 3 products by revenue?"
]

for i, query in enumerate(queries, 1):
    print(f"\n{'─' * 70}")
    print(f"Query {i}: {query}")
    print(f"{'─' * 70}")
    
    response = assistant.query(query)
    print(f"\n💬 Response:\n{response}")

print(f"\n{'═' * 70}")

TEST 1: Q&A Intent - Information Retrieval

──────────────────────────────────────────────────────────────────────
Query 1: What was the total Q1 sales revenue?
──────────────────────────────────────────────────────────────────────


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool: document_reader | Input: Q1 sales revenue | Output: Retrieved 3 document(s)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



💬 Response:
The total Q1 sales revenue for 2024 is $180,000. This figure is derived from the revenue breakdown for the first quarter, which includes the following monthly revenues:

- January: $50,000
- February: $60,000
- March: $70,000

When these amounts are summed, they total $180,000 for Q1 2024 (Source: Document 1).

──────────────────────────────────────────────────────────────────────
Query 2: Who is the CTO of the company?
──────────────────────────────────────────────────────────────────────


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool: document_reader | Input: CTO of the company | Output: Retrieved 3 document(s)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



💬 Response:
The documents provided do not contain any information regarding the Chief Technology Officer (CTO) of the company. The relevant document, "team_structure.txt," is mentioned in the overview, but its content is not included in the retrieved documents. Therefore, I cannot provide the name or details of the CTO based on the available information.

──────────────────────────────────────────────────────────────────────
Query 3: What are the top 3 products by revenue?
──────────────────────────────────────────────────────────────────────


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool: document_reader | Input: top 3 products by revenue | Output: Retrieved 3 document(s)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



💬 Response:
Based on the provided documents, the top 3 products by revenue for Q1 2024 are as follows:

1. **Enterprise Software License (PROD-001)**
   - **Revenue:** $75,000
   - **Description:** Full-featured enterprise software license with unlimited users.
   - **Units Sold:** 15

2. **Cloud Services Subscription - Professional (PROD-002)**
   - **Revenue:** $60,000
   - **Description:** Professional cloud hosting and services package.
   - **Units Sold:** 200

3. **Cloud Services Subscription - Enterprise (PROD-004)**
   - **Revenue:** $50,000
   - **Description:** Enterprise-grade cloud infrastructure with advanced features.
   - **Units Sold:** 50

These products collectively represent significant contributions to the company's revenue in the first quarter of 2024 (Source: Document 2).

══════════════════════════════════════════════════════════════════════


## Test 2: Calculation Intent (Mathematical Operations)

These queries should trigger the calculation agent and use the calculator tool.

In [4]:
print("=" * 70)
print("TEST 2: Calculation Intent - Mathematical Operations")
print("=" * 70)

queries = [
    "Calculate the average monthly sales for Q1 2024",
    "What is the profit margin for Q1?",
    "Calculate the total project budget"
]

for i, query in enumerate(queries, 1):
    print(f"\n{'─' * 70}")
    print(f"Query {i}: {query}")
    print(f"{'─' * 70}")
    
    response = assistant.query(query)
    print(f"\n🧮 Response:\n{response}")

print(f"\n{'═' * 70}")

TEST 2: Calculation Intent - Mathematical Operations

──────────────────────────────────────────────────────────────────────
Query 1: Calculate the average monthly sales for Q1 2024
──────────────────────────────────────────────────────────────────────


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool: document_reader | Input: Q1 2024 sales revenue | Output: Retrieved 3 document(s)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🧮 Response:
expression='Total Q1 Revenue / 3' result='$60,000' explanation='To calculate the average monthly sales for Q1 2024, we take the total revenue for the quarter, which is $180,000, and divide it by the number of months in the quarter (3). Therefore, the average monthly sales are calculated as follows: $180,000 / 3 = $60,000.' units='USD' sources=['/Users/robertishimurasousa/Developer/GitHub/robertishimurasousa/doc_assistant_project/data/financial_overview.txt'] confidence=1.0 timestamp=datetime.datetime(2024, 3, 31, 12, 0, tzinfo=TzInfo(0))

──────────────────────────────────────────────────────────────────────
Query 2: What is the profit margin for Q1?
──────────────────────────────────────────────────────────────────────


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool: document_reader | Input: profit margin Q1 2024 | Output: Retrieved 3 document(s)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🧮 Response:
The profit margin for Q1 2024 can be calculated using the following formula:

\[ \text{Profit Margin} = \left( \frac{\text{Profit}}{\text{Revenue}} \right) \times 100 \]

From the financial overview document, we have the following data for Q1 2024:

- **Total Revenue:** $180,000
- **Total Expenses:** $120,000
- **Profit:** $60,000

Using this information, we can calculate the profit margin:

1. **Calculate Profit Margin:**
   \[
   \text{Profit Margin} = \left( \frac{60,000}{180,000} \right) \times 100 = 33.3\%
   \]

Thus, the profit margin for Q1 2024 is **33.3%** (Source: Document 1). This indicates that for every dollar of revenue, the company retains approximately 33.3 cents as profit after covering its expenses.

──────────────────────────────────────────────────────────────────────
Query 3: Calculate the total project budget
──────────────────────────────────────────────────────────────────────


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool: document_reader | Input: total project budget | Output: Retrieved 3 document(s)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🧮 Response:
expression='Total Project Budget = Website Redesign Budget + Mobile App Development Budget + Data Analytics Dashboard Budget' result='$100,000' explanation='The total project budget is calculated by summing the budgets of all active projects. The budgets are as follows: Website Redesign: $30,000, Mobile App Development: $50,000, and Data Analytics Dashboard: $20,000. Therefore, the total project budget is $30,000 + $50,000 + $20,000 = $100,000.' units='USD' sources=['/Users/robertishimurasousa/Developer/GitHub/robertishimurasousa/doc_assistant_project/data/team_structure.txt'] confidence=1.0 timestamp=datetime.datetime(2024, 3, 31, 12, 0, tzinfo=TzInfo(0))

══════════════════════════════════════════════════════════════════════


## Test 3: Summarization Intent

These queries should route to the summarization agent.

In [5]:
print("=" * 70)
print("TEST 3: Summarization Intent")
print("=" * 70)

queries = [
    "Summarize the Q1 sales report",
    "Give me an overview of the team structure",
    "Summarize the key financial metrics"
]

for i, query in enumerate(queries, 1):
    print(f"\n{'─' * 70}")
    print(f"Query {i}: {query}")
    print(f"{'─' * 70}")
    
    response = assistant.query(query)
    print(f"\n📝 Response:\n{response}")

print(f"\n{'═' * 70}")

TEST 3: Summarization Intent

──────────────────────────────────────────────────────────────────────
Query 1: Summarize the Q1 sales report
──────────────────────────────────────────────────────────────────────


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool: document_reader | Input: Q1 sales report | Output: Retrieved 3 document(s)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



📝 Response:
summary="The Q1 2024 sales report provides a comprehensive overview of the company's performance across various metrics. Total revenue for the quarter reached $180,000, with a monthly breakdown of $50,000 in January, $60,000 in February, and $70,000 in March. The report highlights customer acquisition metrics, regional performance, and product category breakdowns. Key findings indicate a profit of $60,000, resulting in a profit margin of 33.3%. The report also includes recommendations for future sales strategies and identifies growth opportunities in the cloud services and software licenses categories, which contributed significantly to the overall revenue." key_points=['Total Q1 Revenue: $180,000', 'Monthly Breakdown: January - $50,000, February - $60,000, March - $70,000', 'Profit: $60,000 with a profit margin of 33.3%', 'Key product categories: Software Licenses ($106,000), Cloud Services ($110,000), Professional Services ($45,000)', 'Recommendations for improving custo

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool: document_reader | Input: team structure | Output: Retrieved 3 document(s)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



📝 Response:
summary='The company team structure consists of a leadership team and an engineering department, with a total of four members. The leadership team is led by Alice Johnson, the Chief Technology Officer (CTO), who oversees technology strategy and team leadership. The engineering department includes Bob Smith (Senior Software Developer), Carol White (UX/UI Designer), and David Brown (Data Analyst), each with specific roles in ongoing projects. The team is currently working on three active projects: Website Redesign, Mobile App Development, and Data Analytics Dashboard, with a total project budget of $100,000. The team has an average experience of 8.5 years and a satisfaction score of 4.5 out of 5. The company has plans for future hiring and emphasizes training and development for its members.' key_points=['Leadership Team led by CTO Alice Johnson', 'Engineering Department includes Bob Smith, Carol White, and David Brown', 'Total Team Members: 4', 'Average Experience: 8.5 year

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool: document_reader | Input: key financial metrics | Output: Retrieved 3 document(s)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



📝 Response:
summary='The key financial metrics for Q1 2024 indicate a strong performance with total revenue of $180,000, a profit of $60,000, and a profit margin of 33.3%. The revenue showed a monthly increase, with January at $50,000, February at $60,000, and March at $70,000. Total expenses for the quarter were $120,000, primarily driven by salaries and benefits. The company has a projected revenue of $225,000 for Q2, with a focus on maintaining profitability and managing expenses. Key financial ratios include a current ratio of 9.0 and a debt-to-equity ratio of 0.10, indicating a healthy financial position. The company aims for an annual revenue target of $900,000 and plans to invest $210,000 in various growth initiatives.' key_points=['Total Q1 Revenue: $180,000', 'Total Q1 Expenses: $120,000', 'Profit for Q1: $60,000 with a profit margin of 33.3%', 'Projected Q2 Revenue: $225,000', 'Current Ratio: 9.0, indicating strong liquidity', 'Debt-to-Equity Ratio: 0.10, indicating low leve

## Test 4: Conversation Memory

Test that the system maintains context between queries.

In [6]:
print("=" * 70)
print("TEST 4: Conversation Memory - Context Retention")
print("=" * 70)

# First query establishes context
query1 = "What was the revenue in March 2024?"
print(f"\n{'─' * 70}")
print(f"Query 1: {query1}")
print(f"{'─' * 70}")
response1 = assistant.query(query1)
print(f"\n💬 Response 1:\n{response1}")

# Follow-up query uses context
query2 = "How does that compare to January?"
print(f"\n{'─' * 70}")
print(f"Query 2 (follow-up): {query2}")
print(f"{'─' * 70}")
response2 = assistant.query(query2)
print(f"\n💬 Response 2:\n{response2}")

# Another follow-up
query3 = "Calculate the percentage increase"
print(f"\n{'─' * 70}")
print(f"Query 3 (follow-up): {query3}")
print(f"{'─' * 70}")
response3 = assistant.query(query3)
print(f"\n💬 Response 3:\n{response3}")

print(f"\n{'═' * 70}")

TEST 4: Conversation Memory - Context Retention

──────────────────────────────────────────────────────────────────────
Query 1: What was the revenue in March 2024?
──────────────────────────────────────────────────────────────────────


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



💬 Response 1:
The revenue in March 2024 was $70,000. This is part of the total revenue of $180,000 for Q1 2024, which included $50,000 in January and $60,000 in February.

──────────────────────────────────────────────────────────────────────
Query 2 (follow-up): How does that compare to January?
──────────────────────────────────────────────────────────────────────


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



💬 Response 2:
In January 2024, the revenue was $50,000, while in March 2024, the revenue increased to $70,000. This represents a growth of $20,000, or a 40% increase in revenue from January to March.

──────────────────────────────────────────────────────────────────────
Query 3 (follow-up): Calculate the percentage increase
──────────────────────────────────────────────────────────────────────


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool: calculator | Input: ((70000 - 50000) / 50000) * 100 | Output: Result: 40.0
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



💬 Response 3:
expression='((March Revenue - January Revenue) / January Revenue) * 100' result='40.0' explanation='To calculate the percentage increase in revenue from January to March, we use the formula: ((New Value - Old Value) / Old Value) * 100. Here, the New Value is the revenue in March ($70,000) and the Old Value is the revenue in January ($50,000). Thus, the calculation is: ((70,000 - 50,000) / 50,000) * 100 = (20,000 / 50,000) * 100 = 0.4 * 100 = 40%. This indicates a 40% increase in revenue from January to March.' units='%' sources=['calculator'] confidence=0.95 timestamp=datetime.datetime(2024, 3, 31, 12, 0, tzinfo=TzInfo(0))

══════════════════════════════════════════════════════════════════════


## Test 5: Session Statistics

Check session statistics and conversation history.

In [7]:
print("=" * 70)
print("TEST 5: Session Statistics")
print("=" * 70)

stats = assistant.get_stats()

print(f"\n📊 Current Session Statistics:")
print(f"{'─' * 70}")
print(f"Session ID: {stats['current_session_id']}")
print(f"Documents Loaded: {stats['num_documents']}")
print(f"Total Sessions: {stats['num_sessions']}")
print(f"Messages in Current Session: {stats['current_session_messages']}")

print(f"\n📜 Conversation History:")
print(f"{'─' * 70}")
history = assistant.get_session_history()
for i, msg in enumerate(history[-6:], 1):  # Show last 6 messages
    # Message objects are Pydantic models, not dicts - access attributes directly
    if hasattr(msg, 'role'):
        role = msg.role
        content = msg.content if hasattr(msg, 'content') else str(msg)
    elif isinstance(msg, dict):
        role = msg.get('role', 'unknown')
        content = msg.get('content', '')
    else:
        role = 'unknown'
        content = str(msg)
    
    print(f"\n{i}. [{role.upper()}]")
    print(f"   {str(content)[:150]}{'...' if len(str(content)) > 150 else ''}")

print(f"\n{'═' * 70}")

TEST 5: Session Statistics

📊 Current Session Statistics:
──────────────────────────────────────────────────────────────────────
Session ID: 3252ee1e-5cb3-41db-b6b3-e48e2d9608de
Documents Loaded: 6
Total Sessions: 6
Messages in Current Session: 24

📜 Conversation History:
──────────────────────────────────────────────────────────────────────

1. [USER]
   What was the revenue in March 2024?

2. [ASSISTANT]
   The revenue in March 2024 was $70,000. This is part of the total revenue of $180,000 for Q1 2024, which included $50,000 in January and $60,000 in Feb...

3. [USER]
   How does that compare to January?

4. [ASSISTANT]
   In January 2024, the revenue was $50,000, while in March 2024, the revenue increased to $70,000. This represents a growth of $20,000, or a 40% increas...

5. [USER]
   Calculate the percentage increase

6. [ASSISTANT]
   expression='((March Revenue - January Revenue) / January Revenue) * 100' result='40.0' explanation='To calculate the percentage increase in reven

## Test 6: Document Search

Test direct document retrieval.

In [8]:
print("=" * 70)
print("TEST 6: Document Search")
print("=" * 70)

search_queries = [
    "sales revenue",
    "team members",
    "profit margin"
]

for i, search_query in enumerate(search_queries, 1):
    print(f"\n{'─' * 70}")
    print(f"Search {i}: '{search_query}'")
    print(f"{'─' * 70}")
    
    # Use the correct method name and create a Query object
    from src.schemas import Query
    query_obj = Query(text=search_query)
    docs = assistant.retriever.retrieve(query_obj, top_k=3)
    
    print(f"\n📄 Found {len(docs)} relevant documents:")
    for j, doc in enumerate(docs, 1):
        source = doc.metadata.get('filename', 'unknown') if doc.metadata else 'unknown'
        content = doc.content[:200]
        score = doc.score if hasattr(doc, 'score') and doc.score else 0.0
        print(f"\n  {j}. Source: {source} (Score: {score:.2f})")
        print(f"     Preview: {content}...")

print(f"\n{'═' * 70}")

TEST 6: Document Search

──────────────────────────────────────────────────────────────────────
Search 1: 'sales revenue'
──────────────────────────────────────────────────────────────────────

📄 Found 3 relevant documents:

  1. Source: financial_overview.txt (Score: 15.00)
     Preview: Financial Overview 2024

Q1 2024 Financial Summary
--------------------------

Revenue Breakdown
-----------------
Quarter 1 (Q1):
- January: $50,000
- February: $60,000
- Marc...

  2. Source: README.md (Score: 12.00)
     Preview: # Sample Data Files

This directory contains sample documents for testing the Document Assistant.

## Files Overview

### 1. sales_report_q1_2024.txt
**Type**: Text Report
**Size**: ~3 KB
**Content**:...

  3. Source: product_catalog.json (Score: 11.00)
     Preview: {
  "catalog": {
    "name": "Company Product Catalog",
    "version": "2024.1",
    "last_updated": "2024-03-31",
    "currency": "USD"
  },
  "products": [
    {
      "id": "PROD-001",
      "name"...

────

## Test 7: Session Management

Test session save/load functionality.

In [9]:
print("=" * 70)
print("TEST 7: Session Management")
print("=" * 70)

# Save current session
current_session_id = assistant.current_session.session_id
print(f"\n💾 Saving session: {current_session_id}")
assistant.save_session()
print("✅ Session saved")

# List all sessions
print(f"\n📋 Available sessions:")
sessions = assistant.list_sessions()
for i, session_id in enumerate(sessions[-5:], 1):  # Show last 5
    print(f"  {i}. {session_id}")

# Verify session file exists
session_file = project_root / "sessions" / f"{current_session_id}.json"
print(f"\n✅ Session file exists: {session_file.exists()}")
print(f"   Path: {session_file}")

print(f"\n{'═' * 70}")

TEST 7: Session Management

💾 Saving session: 3252ee1e-5cb3-41db-b6b3-e48e2d9608de
✅ Session saved

📋 Available sessions:
  1. 228a7fa1-3527-465e-a19a-3c8c66f3fb50
  2. 251c662c-d121-43d9-bc13-a7f8d9f99993
  3. bdc7d541-94ea-4df9-ba31-0bb24ef86b50
  4. fce72d12-f35e-45aa-90bf-ecbd1b03a9cc
  5. 2e7fe4ae-8b9c-4192-8fa2-11c7864f6ae2

✅ Session file exists: True
   Path: /Users/robertishimurasousa/Developer/GitHub/robertishimurasousa/doc_assistant_project/sessions/3252ee1e-5cb3-41db-b6b3-e48e2d9608de.json

══════════════════════════════════════════════════════════════════════


## Test 8: Custom Query (Your Turn!)

Try your own query here:

In [10]:
# Enter your custom query here
custom_query = "What products does the company offer and what are their prices?"

print("=" * 70)
print("TEST 8: Custom Query")
print("=" * 70)
print(f"\nQuery: {custom_query}")
print(f"{'─' * 70}")

response = assistant.query(custom_query)
print(f"\n💬 Response:\n{response}")

print(f"\n{'═' * 70}")

TEST 8: Custom Query

Query: What products does the company offer and what are their prices?
──────────────────────────────────────────────────────────────────────


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool: document_reader | Input: products and prices | Output: Retrieved 3 document(s)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



💬 Response:
The company offers a variety of products across different categories, each with specific features and pricing. Here is a detailed list of the products along with their prices:

1. **Enterprise Software License**
   - **Category**: Software Licenses
   - **Description**: Full-featured enterprise software license with unlimited users.
   - **Price**: $5,000 (one-time payment)
   - **Units Sold in Q1**: 15
   - **Revenue in Q1**: $75,000

2. **Cloud Services Subscription - Professional**
   - **Category**: Cloud Services
   - **Description**: Professional cloud hosting and services package.
   - **Price**: $299 (monthly)
   - **Units Sold in Q1**: 200
   - **Revenue in Q1**: $60,000

3. **Professional Services - Consulting**
   - **Category**: Professional Services
   - **Description**: Expert consulting services for digital transformation.
   - **Price**: $150 (hourly)
   - **Units Sold in Q1**: 300
   - **Revenue in Q1**: $45,000

4. **Cloud Services Subscription - Enterpri

## Summary Report

Generate a final summary of the testing session.

In [11]:
print("=" * 70)
print("TESTING SESSION SUMMARY")
print("=" * 70)

stats = assistant.get_stats()

print(f"\n✅ All Tests Completed Successfully!\n")
print(f"📊 Final Statistics:")
print(f"{'─' * 70}")
print(f"Session ID: {stats['current_session_id']}")
print(f"Documents Loaded: {stats['num_documents']}")
print(f"Total Messages: {stats['current_session_messages']}")
print(f"Total Sessions: {stats['num_sessions']}")

print(f"\n🧪 Tests Executed:")
print(f"{'─' * 70}")
print(f"  ✓ Test 1: Q&A Intent (Information Retrieval)")
print(f"  ✓ Test 2: Calculation Intent (Mathematical Operations)")
print(f"  ✓ Test 3: Summarization Intent")
print(f"  ✓ Test 4: Conversation Memory")
print(f"  ✓ Test 5: Session Statistics")
print(f"  ✓ Test 6: Document Search")
print(f"  ✓ Test 7: Session Management")
print(f"  ✓ Test 8: Custom Query")

print(f"\n✨ Key Features Demonstrated:")
print(f"{'─' * 70}")
print(f"  • Intent classification (qa, calculation, summarization)")
print(f"  • Multi-agent routing with LangGraph")
print(f"  • Tool usage (calculator, document_reader)")
print(f"  • Structured outputs with Pydantic")
print(f"  • Conversation memory with MemorySaver")
print(f"  • Session persistence")

print(f"\n{'═' * 70}")
print(f"\n🎉 Document Assistant Demo Complete!")
print(f"\n📚 For more information, see:")
print(f"   - README.md - Project overview")
print(f"   - docs/IMPLEMENTATION.md - Technical details")
print(f"   - docs/TESTING_GUIDE.md - Testing instructions")

TESTING SESSION SUMMARY

✅ All Tests Completed Successfully!

📊 Final Statistics:
──────────────────────────────────────────────────────────────────────
Session ID: 3252ee1e-5cb3-41db-b6b3-e48e2d9608de
Documents Loaded: 6
Total Messages: 26
Total Sessions: 6

🧪 Tests Executed:
──────────────────────────────────────────────────────────────────────
  ✓ Test 1: Q&A Intent (Information Retrieval)
  ✓ Test 2: Calculation Intent (Mathematical Operations)
  ✓ Test 3: Summarization Intent
  ✓ Test 4: Conversation Memory
  ✓ Test 5: Session Statistics
  ✓ Test 6: Document Search
  ✓ Test 7: Session Management
  ✓ Test 8: Custom Query

✨ Key Features Demonstrated:
──────────────────────────────────────────────────────────────────────
  • Intent classification (qa, calculation, summarization)
  • Multi-agent routing with LangGraph
  • Tool usage (calculator, document_reader)
  • Structured outputs with Pydantic
  • Conversation memory with MemorySaver
  • Session persistence

════════════════════

## Additional Experiments

Use the cells below for additional experiments:

In [12]:
# Experiment cell 1
# Try different queries here


In [13]:
# Experiment cell 2
# Test edge cases or specific scenarios
